In [1]:
import numpy as np
from numpy import *

In [2]:
x0 = np.array([[1,1.1]])
x1 = np.array([[1.9, 2.2]])
x2 = np.array([[2.9, 3]])
A = np.squeeze(np.array([x0,x1,x2]))
y = np.array([[1,2,3]])
A,y

(array([[1. , 1.1],
        [1.9, 2.2],
        [2.9, 3. ]]), array([[1, 2, 3]]))

In [27]:
"""
min loss(A, w, gamma, y)
subject to ||w||^2 - t <= 0
"""

'\nmin loss(A, w, gamma, y)\nsubject to ||w||^2 <= t\n'

In [39]:
# Loss func loss(x)
def loss_func (w, gamma):
    """
    Objective function: loss / cost function.
    
    Parameters
    ----------
    w array of weights.
    gamma array of intercepts.
    
    Returns
    -------
    The objective function loss(x) evaluated at {w, gamma}.
    
    """
    cost = (1/2)*(w @ A.T + gamma.T - y) @ (w @ A.T + gamma.T - y).T
    return np.squeeze(cost)

In [45]:
def loss_grad (w, gamma): # ??
    """
    Gradient of the objective function (loss / cost function).
    
    Parameters
    ----------
    A matrix with x arrays at the rows.
    w array of weights.
    gamma array of intercepts.
    y array of f(x).
    
    Returns
    -------
    Array of the gradient of the objective function loss(x) 
    evaluated at {w, gamma}.
    
    """
    g_cost = -2* (y - w @ A.T + gamma.T) @ A
    return np.squeeze(g_cost)

In [46]:
def loss_hess (w, gamma):
    """
    Hessian matrix of the objective function (loss / cost function).
    
    Parameters
    ----------
    w array of weights.
    gamma array of intercepts.
    
    Returns
    -------
    Matrix of the Hessian of the objective function loss(x) 
    evaluated at {w, gamma}.
    
    """
    return A.T @ A

In [47]:
def constraint_g (w, t):
    """
    Inequality constraint g
    
    Parameters
    ----------
    w array of weights.
    t value, regularization parameter.
    
    """
    return np.linalg.norm(w)**2 - t

In [48]:
loss_func(w, gamma), loss_grad(w, gamma), loss_hess(w, gamma)

(array(7.015), array([27.  , 29.06]), array([[13.02, 13.98],
        [13.98, 15.05]]))

In [35]:
constraint_g(w, 3)

-0.9999999999999996

In [38]:
from scipy.optimize import minimize
from scipy.optimize import LinearConstraint
from scipy.optimize import Bounds
from scipy.optimize import NonlinearConstraint

In [29]:
nonlinear_constraint = NonlinearConstraint()

5.0

In [ ]:
sol = minimize(loss_func)